In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from textblob import TextBlob, Word
from pandas import DataFrame
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

import gensim
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore')


In [2]:
# read in pickle
with open('result2.pkl', 'rb') as f:
    result = pickle.load(f)

# convert to dataframe and store as pickle
df = pd.DataFrame(result)
df.to_pickle('df.pkl')

# print df head
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,86ff1ea1-0b63-43ce-addc-eb43f6193b3b,Yaseen Yaseen,https://play-lh.googleusercontent.com/a/ALm5wu...,Yaeen Yaeen gg,5,0,None,2022-10-04 20:32:28,None,NaT
1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,2022-10-04 20:32:10,None,NaT
2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,2022-10-04 20:31:21,None,NaT
3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,2022-10-04 20:30:22,None,NaT
4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,2022-10-04 20:29:25,None,NaT


In [3]:
# check fields inside df
df.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt'],
      dtype='object')

In [4]:
# check how many reviews with score of 1
df[df.score == 1].count()

reviewId                14178
userName                14178
userImage               14178
content                 14178
score                   14178
thumbsUpCount           14178
reviewCreatedVersion     9491
at                      14178
replyContent               13
repliedAt                  13
dtype: int64

We will implement functions to clean up the reviews, we will want to remove unnecesary stop words, punctuations @mentions, #hashtags, emojis, emails etc.
We will also perform english detection and spelling correction to ensure the correctedness of the cleaned data 

In [5]:

def remove_punctuations(review: str) -> str:
    words: list[str] = review.split()
    table: dict[int, int] = str.maketrans('', '', string.punctuation)
    stripped: list[str] = [w.translate(table) for w in words]
    return " ".join(stripped)

def normalize(review: str) -> str:
    return review.lower()

def remove_stopwords(review: str, stopwords: set[str]) -> str:
    words: list[str] = review.split()
    words = [w for w in words if not w in stopwords]
    return " ".join(words)


def stemming(review: str) -> str:
    tokens: list[str] = word_tokenize(review)
    porter: PorterStemmer = PorterStemmer()
    stemmed: list[str] = [porter.stem(word) for word in tokens]
    return " ".join(stemmed)

def lemmatize(review: str) -> str:
    lemmatizer = WordNetLemmatizer()
    tokens: list[str] = word_tokenize(review)
    lemmatized: list[str] = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

def depure_data(data):
    
    #remove url with a RegEx
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    #remove email address
    data = re.sub('\S*@\S*\s?', '', data)

    #remove new line characters
    data = re.sub('\s+', ' ', data)

    #remove single quotes
    data = re.sub("\'", "", data)
        
    return data


def detokenize(text):
    #treebank tokenizer uses RegEx to tokenize text as in Penn Treebank
    return TreebankWordDetokenizer().detokenize(text)


def sent_to_words(sentence):
    return gensim.utils.simple_preprocess(str(sentence), deacc=True)
        
def correct_word_spelling(word):
    word = Word(word)
    result = word.correct()
    return result

def correct_sentence(sentence: list[str], spell):
    for j in range(len(sentence)):
        word = sentence[j]

        #use regex to reduce 3 consecutive letters to 2
        reduced_word = re.sub(r'(.)\1+', r'\1\1', word)
        sentence[j] = reduced_word

        #if incorrect spelled, correct word
        if sentence[j] != "tiktok" and len(spell.unknown([sentence[j]])) > 0:
            sentence[j] = correct_word_spelling(sentence[j])

    
    

In [6]:
def clean_review(review: str, stopwords: set[str]) -> str:
    result: str = depure_data(remove_stopwords(normalize(remove_punctuations(review)), stopwords))
    return result

In [7]:
def clean_data(df: DataFrame):
    # df['content'] = df['content'].apply(lambda x: x if is_english(x) else '')
    # df = df[df['content'] != '']
    spell = SpellChecker()
    reviews: list[str] = list(df["content"])
    stop_words: set[str] = set(stopwords.words('english'))
    cleaned_reviews: list[str] = list()
    for i, review in enumerate(reviews):
        try:
            cleaned_review = clean_review(review, stop_words)
            cleaned_review_in_words = sent_to_words(cleaned_review)
            correct_sentence(cleaned_review_in_words, spell)
            cleaned_review = detokenize(cleaned_review_in_words) 
            cleaned_reviews.append(cleaned_review)
            if i%1000==0:
                print(i)
        except:
            cleaned_reviews.append(review)
    df["Clean_texts"] = cleaned_reviews

In [8]:
df = df.loc[(df['content'] != "") & (df['content'] != None)]
for i,r in df.iterrows():
    if r['content'] == "":
        print(r)
    elif r['content'] == None:
        print(r)
        print("content shldnt be none")
df


reviewId                             090a8b76-dd92-49e2-b635-640218b80fda
userName                                         Georgie Georgiev (hacky)
userImage               https://play-lh.googleusercontent.com/a-/ACNPE...
content                                                              None
score                                                                   5
thumbsUpCount                                                           0
reviewCreatedVersion                                               24.9.3
at                                                    2022-09-13 14:10:37
replyContent                                                         None
repliedAt                                                             NaT
Name: 52804, dtype: object
content shldnt be none


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,86ff1ea1-0b63-43ce-addc-eb43f6193b3b,Yaseen Yaseen,https://play-lh.googleusercontent.com/a/ALm5wu...,Yaeen Yaeen gg,5,0,None,2022-10-04 20:32:28,None,NaT
1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,2022-10-04 20:32:10,None,NaT
2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,2022-10-04 20:31:21,None,NaT
3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,2022-10-04 20:30:22,None,NaT
4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,2022-10-04 20:29:25,None,NaT
...,...,...,...,...,...,...,...,...,...,...
99995,b1e4a582-bf47-46ad-a4dc-884468d8eddd,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Best app in pakistan 🇵🇰 tik toker Soham anwar ...,5,1,25.8.5,2022-08-29 23:21:07,None,NaT
99996,95df7741-4bbf-4d04-a60c-03b11f3ba268,Ethio culture,https://play-lh.googleusercontent.com/a-/ACNPE...,Perfect,4,0,25.9.4,2022-08-29 23:20:52,None,NaT
99997,3e6ae27e-d761-4079-98a1-b5e95f296546,Zafar Awan,https://play-lh.googleusercontent.com/a/ALm5wu...,So nice,5,1,25.8.5,2022-08-29 23:19:05,None,NaT
99998,380379d5-7c99-4e0d-a140-6cfd898aa619,Anger Kuiel,https://play-lh.googleusercontent.com/a/ALm5wu...,It's a great app,5,1,25.8.5,2022-08-29 23:18:53,None,NaT


In [9]:
clean_data(df)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [12]:
df.to_pickle('df_english_clean.pkl')


In [ ]:
import re
# return new column "is_english" with True or False
def is_english(text):
    try:
        return bool(re.search('[a-zA-Z]', text))
    except:
        return False
    
df['is_english'] = df['Clean_texts'].apply(is_english)
# remove puncutation
df['Clean_texts'] = df['Clean_texts'].str.replace('[^\w\s]', '')

# save new df as pickle
df.to_pickle('df_english_2.pkl')

# save df to json
df.to_json('df_english_2.json')

# save df to csv
df.to_csv('df_english_2.csv')

C:\Users\ngliw\AppData\Local\Temp\ipykernel_18336\165540621.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')
